In [10]:
import os
import torch
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from joblib import dump, load

In [11]:
train_csv_path = "/home/docker_opr/Datasets/splitted/train/train.csv"

df = pd.read_csv(train_csv_path)
df.head(2)

,Unnamed: 0,timestamp,track,front_cam_ts,back_cam_ts,lidar_ts,northing,easting,tz,qx,qy,qz,qw,back_description,front_description
0,0,1676034260851770,2023-02-10-08-04-19-twilight,1676034260821540,1676034260865870,1676034260851770,-23.839826,-17.655232,-1.474530,0.013051,0.028187,0.003870,0.999510,This image shows a man standing on the stairs ...,This is a picture of a city street with buildi...
1,1,1676034268718093,2023-02-10-08-04-19-twilight,1676034268687989,1676034268708533,1676034268718093,-19.060815,-19.003152,-2.513494,0.017344,0.019243,-0.121944,0.992199,This image shows two people standing outside i...,The image shows a street with buildings on eit...


In [12]:
def train_tfidf_pca(corpus, n_components=100, base_savepath="./opr/datasets/"):
    vectorizer = TfidfVectorizer()
    vectorizer.fit(corpus)
    vectorized_corpus = vectorizer.transform(corpus).toarray()

    pca =PCA(n_components=n_components)
    pca.fit(vectorized_corpus)
    
    vectorizer_savepath = os.path.join(base_savepath, 'vectorizer.joblib')
    pca_savepath = os.path.join(base_savepath, 'pca.joblib')
    dump(vectorizer, vectorizer_savepath) 
    dump(pca, pca_savepath) 

def text_transform(self, text):
    vect_data = self.vectorizer.transform([text]).toarray()
    pca_data = self.pca.transform(vect_data)
    pca_data = torch.tensor(pca_data, dtype=torch.float32)
    return pca_data

In [14]:
corpus = np.hstack((df["back_description"], df["front_description"]))
train_tfidf_pca(corpus)

In [5]:
# vectorizer, pca = train_tfidf_pca(corpus)

In [6]:
base_savepath = "./opr/datasets/"
vectorizer_savepath = os.path.join(base_savepath, 'vectorizer.joblib')
pca_savepath = os.path.join(base_savepath, 'pca.joblib')

In [7]:
base_savepath = "./opr/datasets/"
vectorizer_savepath = os.path.join(base_savepath, 'vectorizer.joblib')
pca_savepath = os.path.join(base_savepath, 'pca.joblib')

vectorizer = load(vectorizer_savepath)
pca = load(pca_savepath)

In [9]:
vectorizer.get_feature_names_out()

array(['00', '10', '100', ..., 'zooms', 'вш', 'медничного'], dtype=object)

In [ ]:
# def train_pipeline(corpus, n_components=300):
#     pipe = Pipeline([('vectorizer', TfidfVectorizer()),
#                      ('pca', PCA(n_components=n_components))])
#     pipe.fit(corpus)
#     return pipe